In [18]:
import numpy as np
from scipy.linalg import svd

def soft_thresholding(x, threshold):
    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

def admm_basis_pursuit(A, b, rho, max_iter=100, tol=1):
    m, n = A.shape
    x = np.zeros(n)
    z = np.zeros(n)
    u = np.zeros(n)
    Atb = A.T @ b
    L = svd(A, full_matrices=False)[1].max()  # Lipschitz constant

    for _ in range(max_iter):
        x_prev = x.copy()

        # Update x
        q = Atb + rho * (z - u)
        if m >= n:
            x = np.linalg.solve(A.T @ A + rho * np.eye(n), q)
        else:
            x = q - A.T @ np.linalg.solve(A @ A.T + rho * np.eye(m), A @ q)

        # Update z
        z = soft_thresholding(x + u, 1 / rho)

        # Update dual variable u
        u = u + x - z

        # Check convergence
        if np.linalg.norm(x - x_prev) / np.linalg.norm(x) < tol:
            break

    return x

# Example usage
A = np.array([[1, 0, 1, 0, 2], 
              [0, 1, 0, 1, 1],
              [2, 0, 3, 0, 1],
              [0, 2, 1, 0, 0],
              [1, 0, 2, 1, 1]])
b = np.array([1, 2, 3, 4, 5])

rho = 1
tol = 1
solution = admm_basis_pursuit(A, b, rho, tol=tol)

print("Solution:", solution)


Solution: [0.03759345 0.95868043 1.14843954 0.86451881 0.13054468]


In [16]:
import numpy as np
from scipy.linalg import svd

def soft_thresholding(x, threshold):
    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

def admm_lasso(X, y, alpha, rho, max_iter=100, tol=1e-4):
    m, n = X.shape
    A = np.linalg.inv(X.T @ X + rho * np.eye(n))
    Xty = X.T @ y

    z = np.zeros(n)
    u = np.zeros(n)

    for _ in range(max_iter):
        z_prev = z.copy()

        # Update z
        q = Xty + rho * (z - u)
        z = A @ q

        # Soft thresholding
        z = soft_thresholding(z, alpha / rho)

        # Update dual variable u
        u = u + z - q

        # Check convergence
        if np.linalg.norm(z - z_prev) / np.linalg.norm(z) < tol:
            break

    return z

# Example usage
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
y = np.array([1, 2, 3, 4])
alpha = 0.1
rho = 1.0

solution = admm_lasso(X, y, alpha, rho)

print("Solution:", solution)


Solution: [6.60835226e+28 7.52512761e+28 8.44190296e+28]
